![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [26]:
# Run the cells below first

In [27]:
!pip install torchmetrics
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [29]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [30]:
# Start coding here
# Use as many cells as you need
train_loader = DataLoader(train_data,batch_size = 64, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = False)

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x



In [31]:
# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=2):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluation loop
def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.numpy())
            all_labels.extend(labels.numpy())
    return predictions, all_labels



In [32]:
# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=2)



Epoch [1/2], Loss: 0.5094
Epoch [2/2], Loss: 0.3246


In [33]:
# Evaluate the model
predictions, all_labels = evaluate_model(model, test_loader)



In [34]:
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall

# Calculate accuracy, precision, and recall
accuracy_metric = MulticlassAccuracy(num_classes=10)
precision_metric = MulticlassPrecision(num_classes=10, average='none')
recall_metric = MulticlassRecall(num_classes=10, average='none')

accuracy = accuracy_metric(torch.tensor(predictions), torch.tensor(all_labels)).item()
precision = precision_metric(torch.tensor(predictions), torch.tensor(all_labels)).tolist()
recall = recall_metric(torch.tensor(predictions), torch.tensor(all_labels)).tolist()


In [35]:
# Store predictions in a list
predictions = predictions



In [36]:
# Output metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.8945
Precision: [0.7951699495315552, 0.9918450713157654, 0.8572949767112732, 0.8936381936073303, 0.7806167602539062, 0.9847715497016907, 0.7591241002082825, 0.9503968358039856, 0.9828628897666931, 0.9526627063751221]
Recall: [0.8889999985694885, 0.9729999899864197, 0.8050000071525574, 0.8989999890327454, 0.8859999775886536, 0.9700000286102295, 0.6240000128746033, 0.9580000042915344, 0.9750000238418579, 0.9660000205039978]
